# Analysis code for the paper "RF shimming in the cervical spinal cord at 7T"

## Data

The data can be downloaded from: https://openneuro.org/datasets/ds004906

The structure of the input dataset is as follows (JSON sidecars are not listed for clarity):
~~~
ds004906
├── CHANGES
├── README
├── dataset_description.json
├── participants.json
├── participants.tsv
├── sub-01
│   ├── anat
│   │   ├── sub-01_acq-CP_T1w.nii.gz
│   │   ├── sub-01_acq-CP_T2starw.nii.gz
│   │   ├── sub-01_acq-CoV_T1w.nii.gz
│   │   ├── sub-01_acq-CoV_T2starw.nii.gz
│   │   ├── sub-01_acq-SAReff_T2starw.nii.gz
│   │   ├── sub-01_acq-patient_T2starw.nii.gz
│   │   ├── sub-01_acq-phase_T2starw.nii.gz
│   │   ├── sub-01_acq-target_T2starw.nii.gz
│   │   ├── sub-01_acq-volume_T2starw.nii.gz
│   └── fmap
│       ├── sub-01_acq-anatCP_TB1TFL.nii.gz
│       ├── sub-01_acq-anatCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-anatSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-anatpatient_TB1TFL.nii.gz
│       ├── sub-01_acq-anatphase_TB1TFL.nii.gz
│       ├── sub-01_acq-anattarget_TB1TFL.nii.gz
│       ├── sub-01_acq-anatvolume_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCP_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-fampSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-famppatient_TB1TFL.nii.gz
│       ├── sub-01_acq-fampphase_TB1TFL.nii.gz
│       ├── sub-01_acq-famptarget_TB1TFL.nii.gz
│       └── sub-01_acq-fampvolume_TB1TFL.nii.gz
├── sub-02
├── sub-03
├── sub-04
└── sub-05
~~~


## Overview of processing pipeline

For each subject:

- Process anat/T2starw (GRE)
  - Segment the spinal cord (SC)
  - Label vertebral levels using existing manual disc labels
  - Create a mask of the cerebrospinal fluid (CSF)
  - Extract the SC/CSF magnitude signal to assess the stability of the flip angle across shim methods
- Process fmap/TFL (flip angle maps)
  - Register each B1 map (CP, CoV, etc.) to the GRE scan
  - Apply the computed warping field to bring the segmentation and vertebral levels to the B1 map
  - Convert the B1 map to nT/V units
  - Extract the B1 map value within the SC

>Slow processes are indicated with the emoji ⏳

In [ ]:
# Necessary imports

from datetime import datetime, timedelta
import os
import re
import json
import subprocess
import glob
import itertools
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import pandas as pd
from scipy.interpolate import interp1d
from scipy.ndimage import uniform_filter1d
from scipy.stats import f_oneway, ttest_rel
import shutil
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.anova import AnovaRM
import statsmodels.api as sm

In [ ]:
# Start timer
start_time = datetime.now()

# Check where we are
!pwd

In [ ]:
# ⚠️ No need to run this cell if you run this notebook locally and already have these dependencies installed.

# Install packages on system
!sudo apt-get update
!sudo apt-get install git-annex

# Install Python libaries
!wget -O requirements.txt https://raw.githubusercontent.com/shimming-toolbox/rf-shimming-7t/main/requirements.txt
!pip install -r requirements.txt

# Install SCT ⏳
!git clone --depth 1 https://github.com/spinalcordtoolbox/spinalcordtoolbox.git
!yes | spinalcordtoolbox/install_sct
os.environ['PATH'] += f":/content/spinalcordtoolbox/bin"

In [ ]:
# Download data and define path variables

!datalad install https://github.com/OpenNeuroDatasets/ds004906.git
os.chdir("ds004906")
!datalad get . # uncomment for production
# !datalad get sub-01/  # debugging
# Get derivatives containing manual labels
!datalad get derivatives

In [ ]:
# Define useful variables

path_data = os.getcwd()
print(f"path_data: {path_data}")
path_labels = os.path.join(path_data, "derivatives", "labels")
path_qc = os.path.join(path_data, "qc")
shim_modes = ["CP", "patient", "volume", "phase", "CoV", "target", "SAReff"]
shim_modes_MPRAGE = ["CP", "CoV"]  # TODO: change variable name PEP8
# shim_modes = ["CP", "CoV"]  # debugging
print(f"shim_modes: {shim_modes}")
subjects = sorted(glob.glob("sub-*"))
print(f"subjects: {subjects}")

# Create output folder
path_results = os.path.join(path_data, 'derivatives', 'results')
os.makedirs(path_results, exist_ok=True)

## Process anat/T2starw (GRE)

In [ ]:
# Run segmentation on GRE scan

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    # The "CoV" RF shimming scenario was chosen as the segmentation baseline due to the more homogeneous signal intensity in the I-->S direction, which results in a better segmentation peformance in the C7-T2 region
    fname_manual_seg = os.path.join(path_labels, subject, "anat", f"{subject}_acq-CoV_T2starw_label-SC_seg.nii.gz")
    if os.path.exists(fname_manual_seg):
        # Manual segmentation already exists. Copy it to local folder
        print(f"{subject}: Manual segmentation found\n")
        shutil.copyfile(fname_manual_seg, f"{subject}_acq-CoV_T2starw_seg.nii.gz")
        # Generate QC report to make sure the manual segmentation is correct
        !sct_qc -i {subject}_acq-CoV_T2starw.nii.gz -s {subject}_acq-CoV_T2starw_seg.nii.gz -p sct_deepseg_sc -qc {path_qc} -qc-subject {subject}
    else:
        # Manual segmentation does not exist. Run automatic segmentation.
        print(f"{subject}: Manual segmentation not found")
        !sct_deepseg_sc -i {subject}_acq-CoV_T2starw.nii.gz -c t2s -qc {path_qc}

In [ ]:
# Copy CSF masks from the derivatives folder

# For more details about how these masks were created, see: https://github.com/shimming-toolbox/rf-shimming-7t/issues/67
for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    fname_manual_seg = os.path.join(path_labels, subject, "anat", f"{subject}_acq-CoV_T2starw_label-CSF_seg.nii.gz")
    if os.path.exists(fname_manual_seg):
        # Manual segmentation already exists. Copy it to local folder
        print(f"{subject}: Manual segmentation found\n")
        shutil.copyfile(fname_manual_seg, f"{subject}_acq-CoV_T2starw_label-CSF_seg.nii.gz")
        # Generate QC report to make sure the manual segmentation is correct
        !sct_qc -i {subject}_acq-CoV_T2starw.nii.gz -s {subject}_acq-CoV_T2starw_label-CSF_seg.nii.gz -p sct_deepseg_sc -qc {path_qc} -qc-subject {subject}
    else:
        # Manual segmentation does not exist. Panic!
        print(f"{subject}: Manual segmentation not found 😱")

In [ ]:
# Crop GRE scan for faster processing and better registration results

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    !sct_crop_image -i {subject}_acq-CoV_T2starw.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop.nii.gz
    !sct_crop_image -i {subject}_acq-CoV_T2starw_seg.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop_seg.nii.gz
    !sct_crop_image -i {subject}_acq-CoV_T2starw_label-CSF_seg.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop_label-CSF_seg.nii.gz

In [ ]:
# Label vertebrae on GRE scan

# Given the low resolution of the GRE scan, the automatic detection of C2-C3 disc is unreliable. Therefore we need to use the manual disc labels that are part of the dataset.
for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    fname_label_discs = os.path.join(path_data, "derivatives", "labels", subject, "anat", f"{subject}_acq-CoV_T2starw_label-discs_dseg.nii.gz")
    !sct_label_utils -i {subject}_acq-CoV_T2starw_crop_seg.nii.gz -disc {fname_label_discs} -o {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz
    # Generate QC report to assess labeled segmentation
    !sct_qc -i {subject}_acq-CoV_T2starw_crop.nii.gz -s {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -p sct_label_vertebrae -qc {path_qc} -qc-subject {subject}

In [ ]:
# Register *_T2starw to CoV_T2starw ⏳

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    for shim_mode in shim_modes:
        # Don't do it for CoV_T2starw
        if shim_mode != 'CoV':
            !sct_register_multimodal -i {subject}_acq-{shim_mode}_T2starw.nii.gz -d {subject}_acq-CoV_T2starw_crop.nii.gz -dseg {subject}_acq-CoV_T2starw_crop_seg.nii.gz -param step=1,type=im,algo=dl -qc {path_qc}

### Verify QC report (GRE segmentation)

Open the quality control (QC) report located under `ds004906/qc/index.html`. Make sure the spinal cord segmentations are correct before resuming the analysis.

>If you run this notebook on Google Colab, you can skip as the QC report cannot easily be viewed from Google Colab. If you *really* want to see the QC report, you can create a cell where you zip the 'qc/' folder, then you can download it on your local station and open the index.html file. 


In [ ]:
# Extract the signal intensity on the GRE scan within the spinal cord between levels C3 and T2 (included), which correspond to the region where RF shimming was prescribed

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    for shim_method in shim_modes:
        # Shim methods are registered to the CoV T2starw scan, so we need to use the added suffix to identify them
        if shim_method == 'CoV':
            file_suffix = 'crop'
        else:
            file_suffix = 'reg'
        fname_result_sc = os.path.join(path_results, f"{subject}_acq-{shim_method}_T2starw_label-SC.csv")
        !sct_extract_metric -i {subject}_acq-{shim_method}_T2starw_{file_suffix}.nii.gz -f {subject}_acq-CoV_T2starw_crop_seg.nii.gz -method wa -vert 3:9 -vertfile {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -perslice 1 -o {fname_result_sc}
        fname_result_csf = os.path.join(path_results, f"{subject}_acq-{shim_method}_T2starw_label-CSF.csv")
        !sct_extract_metric -i {subject}_acq-{shim_method}_T2starw_{file_suffix}.nii.gz -f {subject}_acq-CoV_T2starw_crop_label-CSF_seg.nii.gz -method wa -vert 3:9 -vertfile {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -perslice 1 -o {fname_result_csf}

In [ ]:
# Make figure of CSF/SC signal ratio from T2starw scan

# Go back to root data folder
os.chdir(path_data)

def smooth_data(data, window_size=50):
    """ Apply a simple moving average to smooth the data. """
    return uniform_filter1d(data, size=window_size, mode='nearest')

# Fixed grid for x-axis
x_grid = np.linspace(0, 1, 100)

# z-slices corresponding to levels C3 to T2 on the PAM50 template. These will be used to scale the x-label of each subject.
original_vector = np.array([907, 870, 833, 800, 769, 735, 692, 646])

# Normalize the PAM50 z-slice numbers to the 1-0 range (to show inferior-superior instead of superior-inferior)
min_val = original_vector.min()
max_val = original_vector.max()
normalized_vector = 1 - ((original_vector - min_val) / (max_val - min_val))

# Use this normalized vector as x-ticks
custom_xticks = normalized_vector

# Number of subjects determines the number of rows in the subplot
n_rows = len(subjects)

# Create a figure with multiple subplots
fig, axes = plt.subplots(n_rows, 1, figsize=(10, 6 * n_rows))
font_size = 18

# Check if axes is an array or a single object
if n_rows == 1:
    axes = [axes]

# Data storage for statistics
data_stats = []

# Iterate over each subject and create a subplot
for i, subject in enumerate(subjects):
    ax = axes[i]

    for shim_method in shim_modes:
        # Initialize list to collect data for this shim method
        method_data = []

        # Get signal in SC
        file_csv = os.path.join(path_results, f"{subject}_acq-{shim_method}_T2starw_label-SC.csv")
        df = pd.read_csv(file_csv)
        data_sc = df['WA()']
        data_sc_smoothed = smooth_data(data_sc)

        # Get signal in CSF
        file_csv = os.path.join(path_results, f"{subject}_acq-{shim_method}_T2starw_label-CSF.csv")
        df = pd.read_csv(file_csv)
        data_csf = df['WA()']
        data_csf_smoothed = smooth_data(data_csf)
        
        # Compute ratio
        data_sc_csf_ratio = data_csf_smoothed / data_sc_smoothed

        # Normalize the x-axis to a 0-1 scale for each subject
        x_subject = np.linspace(0, 1, len(data_sc_csf_ratio))

        # Interpolate to the fixed grid
        interp_func = interp1d(x_subject, data_sc_csf_ratio, kind='linear', bounds_error=False, fill_value='extrapolate')
        resampled_data = interp_func(x_grid)

        method_data.append(resampled_data)

        # If there's data for this shim method, plot it and compute stats
        if method_data:
            # Plotting each file's data separately
            for resampled_data in method_data:
                ax.plot(x_grid, resampled_data, label=f"{shim_method}")
        
            # Compute stats on the non-resampled data (to avoid interpolation errors)
            mean_data = np.mean(data_sc_csf_ratio)
            sd_data = np.std(data_sc_csf_ratio)
            data_stats.append([subject, shim_method, mean_data, sd_data])

    # Set custom x-ticks
    ax.set_xticks(custom_xticks)
    ax.set_xticklabels([''] * len(original_vector))

    ax.set_title(f'{subject}', fontsize=font_size)
    ax.set_ylabel('CSF/Cord T2starw signal ratio', fontsize=font_size)
    ax.tick_params(axis='y', which='major', labelsize=font_size-4)

    # Add legend only to the first subplot
    if i == 0:
        ax.legend(fontsize=font_size)

    ax.grid(True)

# Adjust the layout so labels and titles do not overlap
plt.tight_layout()
plt.show()

In [ ]:
# Perform statistics

# Convert to DataFrame and save to CSV
df_stats = pd.DataFrame(data_stats, columns=['Subject', 'Shim_Mode', 'Average', 'Standard_Deviation'])
df_stats.to_csv(os.path.join(path_results, 'stats_t2starw.csv'), index=False)

# Compute statistics across subjects
grouped_means = df_stats.groupby('Shim_Mode').agg({'Average': ['mean', 'std'], 'Standard_Deviation': ['mean', 'std']})

# Format mean ± standard deviation
grouped_means['Average_formatted'] = grouped_means['Average']['mean'].map("{:.2f}".format) + " ± " + grouped_means['Average']['std'].map("{:.2f}".format)
grouped_means['Standard_Deviation_formatted'] = grouped_means['Standard_Deviation']['mean'].map("{:.2f}".format) + " ± " + grouped_means['Standard_Deviation']['std'].map("{:.2f}".format)

# Drop multi-level index and only keep formatted columns
grouped_means = grouped_means.drop(columns=['Average', 'Standard_Deviation'])
grouped_means.columns = ['Average', 'Standard_Deviation']  # Rename columns for clarity
grouped_means.reset_index().to_csv(os.path.join(path_results, 'stats_t2starw_average_across_subjects.csv'), index=False)

# Perform Repeated Measures ANOVA
aovrm = AnovaRM(df_stats, 'Average', 'Subject', within=['Shim_Mode'])
res = aovrm.fit()
print(res.summary())

# Perform Post Hoc paired t-tests

# Filter out subjects that don't have all shim modes
shim_modes = df_stats['Shim_Mode'].unique()
valid_subjects = df_stats.groupby('Subject').filter(lambda x: all(mode in x['Shim_Mode'].values for mode in shim_modes))

# Pairwise comparisons
pairs = list(itertools.combinations(shim_modes, 2))
p_values = []

for pair in pairs:
    data1 = valid_subjects[valid_subjects['Shim_Mode'] == pair[0]]['Average']
    data2 = valid_subjects[valid_subjects['Shim_Mode'] == pair[1]]['Average']
    _, p_value = ttest_rel(data1, data2)
    p_values.append(p_value)

# Function for Benjamini-Hochberg FDR correction
def benjamini_hochberg(p_values):
    n = len(p_values)
    sorted_p_values = np.sort(p_values)
    sorted_index = np.argsort(p_values)
    adjusted_p_values = np.zeros(n)

    for i, p in enumerate(sorted_p_values):
        adjusted_p_values[sorted_index[i]] = min(p * n / (i + 1), 1)

    return adjusted_p_values

# Applying Benjamini-Hochberg FDR correction
adjusted_p_values = benjamini_hochberg(p_values)

# Creating a summary DataFrame
comparison_results = pd.DataFrame({'Pair': pairs, 'P-Value': p_values, 'Adjusted P-Value': adjusted_p_values})

# Save to CSV
comparison_results.to_csv(os.path.join(path_results, 'stats_t2starw_paired_ttest.csv'), index=False)

print(f"Paired t-tests:\n{comparison_results}")

## Process fmap/TFL (flip angle maps)

In [ ]:
# Register TFL flip angle maps to the GRE scan ⏳

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        !sct_register_multimodal -i {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -d ../anat/{subject}_acq-CoV_T2starw_crop.nii.gz -dseg ../anat/{subject}_acq-CoV_T2starw_crop_seg.nii.gz -param step=1,type=im,algo=slicereg,metric=CC -qc {path_qc}

### Verify QC report (B1maps to GRE registration)

Open the QC report located under `ds004906/qc/index.html`. Make sure the registration are correct before resuming the analysis.

In [ ]:
# Warping spinal cord segmentation and vertebral level to each flip angle map

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        # Use linear interpolation to preserve partial volume information (needed when extracting values along the cord)
        !sct_apply_transfo -i ../anat/{subject}_acq-CoV_T2starw_crop_seg.nii.gz -d {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -w warp_{subject}_acq-CoV_T2starw_crop2{subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -x linear -o {subject}_acq-anat{shim_mode}_TB1TFL_seg.nii.gz
        # Use nearest neighbour (nn) interpolation because we are dealing with non-binary discrete labels
        !sct_apply_transfo -i ../anat/{subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -d {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -w warp_{subject}_acq-CoV_T2starw_crop2{subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -x nn -o {subject}_acq-anat{shim_mode}_TB1TFL_seg_labeled.nii.gz

In [ ]:
# Convert the flip angle maps to B1+ efficiency maps [nT/V] (inspired by code from Kyle Gilbert)
# The approach consists in calculating the B1+ efficiency using a 1ms, pi-pulse at the acquisition voltage,
# then scale the efficiency by the ratio of the measured flip angle to the requested flip angle in the pulse sequence.

GAMMA = 2.675e8;  # [rad / (s T)]
requested_fa = 90  # saturation flip angle -- hard-coded in sequence

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        # Fetch the reference voltage from the JSON sidecar to the TFL B1map sequence
        with open(f"{subject}_acq-famp{shim_mode}_TB1TFL.json", "r") as f:
            metadata = json.load(f)
            ref_voltage = metadata.get("TxRefAmp", "N/A")
            print(f"ref_voltage [V]: {ref_voltage} ({subject}_acq-famp{shim_mode}_TB1TFL)")

        # Open flip angle map with nibabel
        nii = nib.load(f"{subject}_acq-famp{shim_mode}_TB1TFL.nii.gz")
        acquired_fa = nii.get_fdata()

        # Siemens maps are in units of flip angle * 10 (in degrees)
        acquired_fa = acquired_fa / 10

        # Account for the power loss between the coil and the socket. That number was given by Siemens.
        voltage_at_socket = ref_voltage * 10 ** -0.095

        # Compute B1 map in [T/V]
        b1_map = (acquired_fa / requested_fa) * (np.pi / (GAMMA * 1e-3 * voltage_at_socket))

        # Convert to [nT/V]
        b1_map = b1_map * 1e9

        # Save as NIfTI file
        nii_b1 = nib.Nifti1Image(b1_map, nii.affine, nii.header)
        nib.save(nii_b1, f"{subject}_acq-{shim_mode}_TB1map.nii.gz")

In [ ]:
# Extract B1+ value along the spinal cord between levels C3 and T2 (included)

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        !sct_extract_metric -i {subject}_acq-{shim_mode}_TB1map.nii.gz -f {subject}_acq-anat{shim_mode}_TB1TFL_seg.nii.gz -method wa -vert 3:9 -vertfile {subject}_acq-anat{shim_mode}_TB1TFL_seg_labeled.nii.gz -perslice 1 -o TB1map_{shim_mode}.csv

In [ ]:
# Make figure of B1+ values along the spinal cord across shim methods

# Go back to root data folder
os.chdir(os.path.join(path_data))

def smooth_data(data, window_size=20):
    """ Apply a simple moving average to smooth the data. """
    return uniform_filter1d(data, size=window_size, mode='nearest')

# Fixed grid for x-axis
x_grid = np.linspace(0, 1, 100)

# z-slices corresponding to levels C3 to T2 on the PAM50 template. These will be used to scale the x-label of each subject.
original_vector = np.array([907, 870, 833, 800, 769, 735, 692, 646])

# Normalize the PAM50 z-slice numbers to the 1-0 range (to show inferior-superior instead of superior-inferior)
min_val = original_vector.min()
max_val = original_vector.max()
normalized_vector = 1 - ((original_vector - min_val) / (max_val - min_val))

# Use this normalized vector as x-ticks
custom_xticks = normalized_vector

# Number of subjects determines the number of rows in the subplot
n_rows = len(subjects)

# Create a figure with multiple subplots
fig, axes = plt.subplots(n_rows, 1, figsize=(10, 6 * n_rows))
font_size = 18

# Check if axes is an array or a single object
if n_rows == 1:
    axes = [axes]

# Data storage for statistics
data_stats = []

# Iterate over each subject and create a subplot
for i, subject in enumerate(subjects):
    ax = axes[i]
    
    os.chdir(os.path.join(path_data, subject, "fmap"))

    for shim_mode in shim_modes:
        # Initialize list to collect data for this shim method
        method_data = []

        for file_path in glob.glob(f"TB1map_*{shim_mode}*.csv"):
            df = pd.read_csv(file_path)
            wa_data = df['WA()']

            # Normalize the x-axis to a 0-1 scale for each subject
            x_subject = np.linspace(0, 1, len(wa_data))

            # Interpolate to the fixed grid
            interp_func = interp1d(x_subject, wa_data, kind='linear', bounds_error=False, fill_value='extrapolate')
            resampled_data = interp_func(x_grid)

            # Apply smoothing
            smoothed_data = smooth_data(resampled_data)

            method_data.append(smoothed_data)

            # If there's data for this shim method, plot it
            if method_data:
                # Plotting each file's data separately
                for resampled_data in method_data:
                    ax.plot(x_grid, resampled_data, label=f"{shim_mode}")

                # Compute stats on the non-resampled data (to avoid interpolation errors)
                mean_data = np.mean(wa_data)
                sd_data = np.std(wa_data)
                data_stats.append([subject, shim_mode, mean_data, sd_data])

    # Set custom x-ticks
    ax.set_xticks(custom_xticks)
    ax.set_xticklabels([''] * len(original_vector))

    ax.set_title(f'{subject}', fontsize=font_size)
    ax.set_ylabel('B1+ efficiency [nT/V]', fontsize=font_size)
    ax.tick_params(axis='y', which='major', labelsize=font_size-4)

    # Add legend only to the first subplot
    if i == 0:
        ax.legend(fontsize=font_size)

    ax.grid(True)

# Adjust the layout so labels and titles do not overlap
plt.tight_layout()
plt.show()

In [ ]:
# Perform statistics

# Convert to DataFrame and save to CSV
df_stats = pd.DataFrame(data_stats, columns=['Subject', 'Shim_Mode', 'Average', 'Standard_Deviation'])
df_stats.to_csv(os.path.join(path_results, 'stats_b1plus.csv'), index=False)

# Compute statistics across subjects
grouped_means = df_stats.groupby('Shim_Mode').agg({'Average': ['mean', 'std'], 'Standard_Deviation': ['mean', 'std']})

# Format mean ± standard deviation
grouped_means['Average_formatted'] = grouped_means['Average']['mean'].map("{:.2f}".format) + " ± " + grouped_means['Average']['std'].map("{:.2f}".format)
grouped_means['Standard_Deviation_formatted'] = grouped_means['Standard_Deviation']['mean'].map("{:.2f}".format) + " ± " + grouped_means['Standard_Deviation']['std'].map("{:.2f}".format)

# Drop multi-level index and only keep formatted columns
grouped_means = grouped_means.drop(columns=['Average', 'Standard_Deviation'])
grouped_means.columns = ['Average', 'Standard_Deviation']  # Rename columns for clarity
grouped_means.reset_index().to_csv(os.path.join(path_results, 'stats_b1plus_average_across_subjects.csv'), index=False)

# Perform Repeated Measures ANOVA
aovrm = AnovaRM(df_stats, 'Average', 'Subject', within=['Shim_Mode'])
res = aovrm.fit()
print(res.summary())

# Perform Post Hoc paired t-tests

# Filter out subjects that don't have all shim modes
shim_modes = df_stats['Shim_Mode'].unique()
valid_subjects = df_stats.groupby('Subject').filter(lambda x: all(mode in x['Shim_Mode'].values for mode in shim_modes))

# Pairwise comparisons
pairs = list(itertools.combinations(shim_modes, 2))
p_values = []

for pair in pairs:
    data1 = valid_subjects[valid_subjects['Shim_Mode'] == pair[0]]['Average']
    data2 = valid_subjects[valid_subjects['Shim_Mode'] == pair[1]]['Average']
    _, p_value = ttest_rel(data1, data2)
    p_values.append(p_value)

# Function for Benjamini-Hochberg FDR correction
def benjamini_hochberg(p_values):
    n = len(p_values)
    sorted_p_values = np.sort(p_values)
    sorted_index = np.argsort(p_values)
    adjusted_p_values = np.zeros(n)

    for i, p in enumerate(sorted_p_values):
        adjusted_p_values[sorted_index[i]] = min(p * n / (i + 1), 1)

    return adjusted_p_values

# Applying Benjamini-Hochberg FDR correction
adjusted_p_values = benjamini_hochberg(p_values)

# Creating a summary DataFrame
comparison_results = pd.DataFrame({'Pair': pairs, 'P-Value': p_values, 'Adjusted P-Value': adjusted_p_values})

# Save to CSV
comparison_results.to_csv(os.path.join(path_results, 'stats_b1plus_paired_ttest.csv'), index=False)

print(f"Paired t-tests:\n{comparison_results}")

In [ ]:
# Prepare RF shimming mask for figure

# Select subject to show
subject = 'sub-05'
os.chdir(os.path.join(path_data, subject, "fmap"))

# Create RF shimming mask from the segmentation (re-create the live RF shimming procedure)
file_mask = f"{subject}_acq-anatCP_TB1TFL_mask-shimming.nii.gz"
!sct_maths -i {subject}_acq-anatCP_TB1TFL_seg_labeled.nii.gz -thr 3 -uthr 9 -o {file_mask}
!sct_maths -i {file_mask} -bin 1 -o {file_mask}
!sct_create_mask -i {subject}_acq-anatCP_TB1TFL.nii.gz -p centerline,{file_mask} -size 28mm -f cylinder -o {file_mask}

# Dilate mask
!sct_maths -i {file_mask} -dilate 2 -shape disk -dim 2 -o {subject}_acq-anatCP_TB1TFL_mask-shimming_dil.nii.gz
# Subtract dilated mask with mask to get the edge
!sct_maths -i {subject}_acq-anatCP_TB1TFL_mask-shimming_dil.nii.gz -sub {file_mask} -o {file_mask}

In [ ]:
# Create figure of B1+ maps

# Select subject to show
subject = 'sub-05'

# Defining crop limits for resulting figure
xmin = 20
xmax = 110
ymin = 20
ymax = 150

# Defining dynamic range
dynmin = 5 
dynmax = 30

# Create a figure with multiple subplots
fig, axes = plt.subplots(2, 4, figsize=(8, 6))
font_size = 14
axes=axes.flatten()

# Plotter loop (avoiding the generation of an ungly 4D data structure)
for i,shim_mode in enumerate(shim_modes):
    if i==0: # Grabbing the CP mode anat to use for displaying the SC and cropping noise for the B1+ map
        
        # Load data
        CP_anat=nib.load(f"{subject}_acq-anat{shim_mode}_TB1TFL.nii.gz")
        CP_SC=nib.load(file_mask)
        CP_nTpV=nib.load(f"{subject}_acq-{shim_mode}_TB1map.nii.gz")
        
        # Defining mask based on the magnitude image intensity threshold
        cslice=CP_anat.shape[2] // 2 -2 #shows the SC seg best
        threshold=300
        mask=CP_anat.get_fdata()[xmin:xmax,ymin:ymax, cslice]
        mask=np.where(mask > threshold, 1, 0)
        
        # Cropping anat, SC, B1+
        CP_anat=CP_anat.get_fdata()[xmin:xmax,ymin:ymax, cslice]
        CP_anat=CP_anat*mask
        CP_SC=CP_SC.get_fdata()[xmin:xmax,ymin:ymax, cslice]
        CP_nTpV=CP_nTpV.get_fdata()[xmin:xmax,ymin:ymax, cslice]
        CP_nTpV=CP_nTpV*mask
        
        # All opacity overalys look ugly: workaround, set the anat slice to a max value where the segmentation exists
        CP_anat[CP_SC>0.5]=4000;
        
        # Plotting anat overlayed with SC
        splot=axes[i]
        splot.imshow((CP_anat.T), cmap='gray', origin='lower',vmin=0,vmax=2000)#, interpolation='spline36')
        splot.set_title('SC overlay', size=font_size)
        splot.axis('off')
        
        # Plotting the B1+ map
        splot=axes[i+1]
        splot.imshow((CP_nTpV.T), cmap='viridis', origin='lower',vmin=dynmin,vmax=dynmax)#, interpolation='spline36')
        splot.set_title(shim_mode, size=font_size)
        splot.axis('off')

        
    else:
        # Load data
        B1map=nib.load(f"{subject}_acq-{shim_mode}_TB1map.nii.gz")
        B1map=B1map.get_fdata()[xmin:xmax,ymin:ymax, cslice]
        B1map=B1map*mask
        
        # Plot
        splot=axes[i+1]
        im = splot.imshow((B1map.T), cmap='viridis', origin='lower',vmin=dynmin,vmax=dynmax)#, interpolation='spline36')
        splot.set_title(shim_mode, size=font_size)
        splot.axis('off')

plt.tight_layout()
plt.subplots_adjust(wspace=0.1, hspace=0, right=0.9)

# Colorbar
# Assume that the colorbar should start at the bottom of the lower row of subplots and
# extend to the top of the upper row of subplots
cbar_bottom = 0.06  # This might need adjustment
cbar_height = 0.83  # This represents the total height of both rows of subplots
cbar_ax = fig.add_axes([0.93, cbar_bottom, 0.03, cbar_height])
cbar = plt.colorbar(im, cax=cbar_ax)

# cbar_ax = fig.add_axes([0.95, 0.5, 0.04, 0.4])
# cbar = plt.colorbar(im, cax=cbar_ax)
cbar_ax.set_title('nT/V', size=12)
plt.show


In [ ]:
# Indicate duration of data processing

end_time = datetime.now()
total_time = (end_time - start_time).total_seconds()

# Convert seconds to a timedelta object
total_time_delta = timedelta(seconds=total_time)

# Format the timedelta object to a string
formatted_time = str(total_time_delta)

# Pad the string representation if less than an hour
formatted_time = formatted_time.rjust(8, '0')

print(f"Total Runtime [hour:min:sec]: {formatted_time}")